In [17]:
import torch
from torch import nn
from d2l import torch as d2l

## 输入表示
下面的`get_tokens_and_segments`将一个句子或两个句子作为输入，然后返回BERT输入序列的标记及其相应的片段索引。

In [18]:
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>'] # 这是第一个句子
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2) # 标记第一个句子的segment索引
    if tokens_b is not None: # 如果有第二个句子
        tokens += tokens_b + ['<sep>'] # 将第二个句子添加到tokens中
        segments += [1] * (len(tokens_b) + 1) # 标记第二个句子的segment索引
    return tokens, segments

下面的`BERTEncoder`类类似于 :numref:`sec_transformer`中实现的`TransformerEncoder`类。与`TransformerEncoder`不同，`BERTEncoder`使用片段嵌入和可学习的位置嵌入。


In [19]:
class BERTEncoder(nn.Module):
    """BERT编码器"""

    def __init__(self,
                 vocab_size, # 词汇表大小
                 num_hiddens, # 隐藏层大小, 也就是每个词向量的长度, 也是自多头注意力的hidden维度
                 norm_shape, # Layer Norm 的输入
                 ffn_num_input, # FFN的输入大小
                 ffn_num_hiddens, 
                 num_heads, # 注意力头数
                 num_layers, # 一共要多少个Encoder块
                 dropout,
                 max_len=1000, # 可学习的位置编码的长度, 应该要大于输入语句的最大程度
                 key_size=768, # Key的大小, Query的大小, Value的大小都是一样的, 因为Encoder里面用的是自注意力
                 query_size=768, 
                 value_size=768,
                 **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        # token_embedding 将词元编码为词向量
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        # 这是片段嵌入, 也就是区分两个句子的
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        # 位置嵌入: 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))
        # 下面是网络的主题部分: Transformer编码器
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", d2l.EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))

    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        # 先进行词嵌入和segment embedding
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        # 再进行位置嵌入
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        # 将处理好的输入放进Transformer编码器中
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X


假设词表大小为10000，为了演示`BERTEncoder`的前向推断，让我们创建一个实例并初始化它的参数。


In [20]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

下面试一试

In [33]:
# 创建两个长度为8的输入, 每个元素是一个还没有embedding的词元
tokens = torch.randint(0, vocab_size, (2, 8))
# 这是segment embedding, 
# 第一个输入是 <cls> A B <sep> C D E <sep>
# 第二个输入是 <cls> A <sep> B C D E <sep>
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
# 然后把输入放进BERT encoder, 注意我们没有给valid_lens, 表示全部都是有效的
encoded_X = encoder(tokens, segments, None)
# 看看输出的形状
encoded_X.shape
# 输出了输入序列的"高级编码", 前两个维度是不变的, 最后一个维度变成了BERT的hidden_size, 也就是BERT内部的特征大小

torch.Size([2, 8, 768])

## 预训练任务

### 掩蔽语言模型（Masked Language Modeling）
我们实现了下面的`MaskLM`类来预测BERT预训练的掩蔽语言模型任务中的掩蔽标记。

预测使用单隐藏层的多层感知机（`self.mlp`）。

在前向推断中，它需要两个输入：`BERTEncoder`的编码结果和用于预测的词元位置。输出是这些位置的预测结果。


In [6]:
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""

    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(
            nn.Linear(num_inputs, num_hiddens),
            nn.ReLU(),
            nn.LayerNorm(num_hiddens),  # 这里还有一个LayerNorm层
            nn.Linear(num_hiddens, vocab_size))

    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1]  # 一共有多少个位置需要预测
        pred_positions = pred_positions.reshape(-1)  # 全部排成一列
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)  # batch的序号
        # 假设batch_size=4，num_pred_positions=2
        # 那么batch_idx是np.array（[0,0,1,1,2,2,3,3]）
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions) # 等价于 np.repeat([0,1,2,3], 2)
        # 取出X里面需要预测的位置
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        # 进行预测
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat


预测一下

In [11]:
mlm = MaskLM(vocab_size, num_hiddens)
# 下面是两个句子里面需要进行预测的位置
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
# 这里的输入是上面BERT encoder的输出
mlm_Y_hat = mlm(encoded_X, mlm_positions)
# 输出的形状应该是: (batch_size, num_pred_positions, vocab_size)
mlm_Y_hat.shape
# (第几个输入, 这个输入需要预测的词元, 词元的预测值, 也就是vocab里面的编号)

torch.Size([2, 3, 10000])

通过掩码下的预测词元`mlm_Y`的真实标签`mlm_Y_hat`，我们可以计算在BERT预训练中的遮蔽语言模型任务的交叉熵损失。


In [12]:
# 假设下面是真实的词元编号
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
# 使用交叉熵, reduction='none'的意思是为每个位置单独计算交叉熵, 不求和或取平均
loss = nn.CrossEntropyLoss(reduction='none')
# 得全部拉平再进行交叉熵计算
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

### 下一句预测（Next Sentence Prediction）
~~这个的实现很简单, 一个全连接层就完了~~

这里依然使用一个单隐藏层的MLP来进行预测, 只是需要注意输入的只有`<cls>`这一个位置的特征.

这里的实现有点搞, 我重构了一下, 把BERT里面的有个hidden拿了进来

In [39]:
# @save
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""

    def __init__(self, num_inputs,  num_hiddens, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        print("num_inputs:", num_inputs)

        self.output = nn.Sequential(
            nn.Linear(num_inputs, num_hiddens),
            nn.Tanh(),
            nn.Linear(num_hiddens, 2))

    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        return self.output(X)


我们可以看到，`NextSentencePred`实例的前向推断返回每个BERT输入序列的二分类预测。


In [41]:
nsp_input = encoded_X[:, 0, :]
print("nsp_input", nsp_input.shape)
num_hiddens = 768
# NSP的输入形状:(batchsize，num_hiddens) 只需要输入<cls>的特征
nsp = NextSentencePred(nsp_input.shape[-1], num_hiddens)
nsp_Y_hat = nsp(nsp_input)
nsp_Y_hat.shape

nsp_input torch.Size([2, 768])
num_inputs: 768


torch.Size([2, 2])

还可以计算两个二元分类的交叉熵损失。


In [43]:
nsp_y = torch.tensor([0, 1])
# 这个loss就是前面的CE, 注意是reduction='none'的
nsp_l = loss(nsp_Y_hat, nsp_y)
print(nsp_l.shape)
print(nsp_l)

torch.Size([2])
tensor([1.1009, 0.5678], grad_fn=<NllLossBackward0>)


值得注意的是，上述两个预训练任务中的所有标签都可以从预训练语料库中获得，而无需人工标注。原始的BERT已经在图书语料库 :cite:`Zhu.Kiros.Zemel.ea.2015`和英文维基百科的连接上进行了预训练。这两个文本语料库非常庞大：它们分别有8亿个单词和25亿个单词。

## 整合代码

在预训练BERT时，最终的损失函数是掩蔽语言模型损失函数和下一句预测损失函数的线性组合。现在我们可以通过实例化三个类`BERTEncoder`、`MaskLM`和`NextSentencePred`来定义`BERTModel`类。前向推断返回编码后的BERT表示`encoded_X`、掩蔽语言模型预测`mlm_Y_hat`和下一句预测`nsp_Y_hat`。


In [ ]:
# @save
class BERTModel(nn.Module):
    """BERT模型"""

    def __init__(self,
                 vocab_size,
                 num_hiddens,
                 norm_shape,
                 ffn_num_input,
                 ffn_num_hiddens,
                 num_heads,
                 num_layers,
                 dropout,

                 max_len=1000,
                 #  这里设置默认大小768有点误导, 768是BERT base版本的hidden_size
                 #  其实这里的维度要和前面的hidden_size对应起来,
                 key_size=768,
                 query_size=768,
                 value_size=768,

                 mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
                                   ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                                   dropout, max_len=max_len, key_size=key_size,
                                   query_size=query_size, value_size=value_size)

        # mlm的hidden_size是768, 但是可以取不一样的
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features, num_hiddens)

    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        nsp_Y_hat = self.nsp(encoded_X[:, 0, :])
        return encoded_X, mlm_Y_hat, nsp_Y_hat


In [24]:
encoded_X[:, 0, :]
print(encoded_X.shape)
print(encoded_X[:, 0, :].shape)

torch.Size([2, 8, 768])
torch.Size([2, 768])


## 小结

* word2vec和GloVe等词嵌入模型与上下文无关。它们将相同的预训练向量赋给同一个词，而不考虑词的上下文（如果有的话）。它们很难处理好自然语言中的一词多义或复杂语义。
* 对于上下文敏感的词表示，如ELMo和GPT，词的表示依赖于它们的上下文。
* ELMo对上下文进行双向编码，但使用特定于任务的架构（然而，为每个自然语言处理任务设计一个特定的体系架构实际上并不容易）；而GPT是任务无关的，但是从左到右编码上下文。
* BERT结合了这两个方面的优点：它对上下文进行双向编码，并且需要对大量自然语言处理任务进行最小的架构更改。
* BERT输入序列的嵌入是词元嵌入、片段嵌入和位置嵌入的和。
* 预训练包括两个任务：掩蔽语言模型和下一句预测。前者能够编码双向上下文来表示单词，而后者则显式地建模文本对之间的逻辑关系。

## 练习

1. 为什么BERT成功了？
1. 在所有其他条件相同的情况下，掩蔽语言模型比从左到右的语言模型需要更多或更少的预训练步骤来收敛吗？为什么？
1. 在BERT的原始实现中，`BERTEncoder`中的位置前馈网络（通过`d2l.EncoderBlock`）和`MaskLM`中的全连接层都使用高斯误差线性单元（Gaussian error linear unit，GELU） :cite:`Hendrycks.Gimpel.2016`作为激活函数。研究GELU与ReLU之间的差异。
